In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#For interactive plots
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [26]:
#load the data
df = pd.read_csv('data/fao_data_fertilizers_data.csv')


beehives = df.loc[df['category']=='beehives']
beehives.head()


,country_or_area,element_code,element,year,unit,value,value_footnotes,category
8042,Africa +,11,Stocks,2007.0,No,16199398.0,A,beehives
8043,Africa +,11,Stocks,2006.0,No,16214756.0,A,beehives
8044,Africa +,11,Stocks,2005.0,No,15362910.0,A,beehives
8045,Africa +,11,Stocks,2004.0,No,15803095.0,A,beehives
8046,Africa +,11,Stocks,2003.0,No,15465850.0,A,beehives


Separate the different countries and areas in two different lists

In [27]:
countries_and_areas=beehives['country_or_area'].unique()
countries=countries_and_areas[[x.find('+')==-1 for x in countries_and_areas]]
weird_stuff=countries[-5:]
countries=countries[:-5]

areas=countries_and_areas[[x.find('+')!=-1 for x in countries_and_areas]]
print('Countries: ')
print(countries.tolist())
print('\nAreas: ')
print(areas.tolist())
print('\nWeird stuff: ')
print(weird_stuff.tolist())

Countries: 
['Albania', 'Algeria', 'Angola', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Belarus', 'Belgium', 'Belgium-Luxembourg', 'Belize', 'Bolivia', 'Bosnia and Herzegovina', 'Brazil', 'Bulgaria', 'Burundi', 'Cameroon', 'Canada', 'Central African Republic', 'Chad', 'Chile', 'China', 'China, mainland', 'Colombia', 'Cook Islands', 'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czech Republic', 'Czechoslovakia', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Estonia', 'Ethiopia', 'Ethiopia PDR', 'Fiji', 'Finland', 'France', 'French Polynesia', 'Georgia', 'Germany', 'Greece', 'Greenland', 'Guadeloupe', 'Guam', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Honduras', 'Hungary', 'Iran, Islamic Republic of', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jordan', 'Kenya', 'Korea, Republic of', 'Kyrgyzstan', 'Latvia', 'Lebanon', 'Libyan Arab Jamahiriya', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Madagascar', 'Mali', 'Martinique', 'Mexico', 'Moldova', 'M

I found some weird stuff as you can see, I don't know what it is. We should probably try to find out.

We can split again the areas list: some areas are geographical others depend on the developping situation of the country.

I'll check for the weird stuff first:

In [29]:
beehives.loc[beehives['country_or_area']==weird_stuff[0]]

,country_or_area,element_code,element,year,unit,value,value_footnotes,category
14903,fnSeqID,Footnote,NaN,NaN,NaN,NaN,NaN,beehives


Ok, so there is no data in the weird stuff, let's forget about it and clean the data.

In [30]:
beehives = beehives[[name not in weird_stuff.tolist() for name in beehives['country_or_area']]]

Let's split again our areas

In [39]:
geo_areas=areas[[x.lower().find('developing')==-1 and x.lower().find('countries')==-1 for x in areas]]
status_areas=areas[[x.lower().find('developing')!=-1 or x.lower().find('countries')!=-1 for x in areas]]

In [40]:
@interact
def show_beehives_for_country(country=countries):
    beehives_country=beehives.loc[beehives['country_or_area']==country]
    plt.plot(beehives_country['year'],beehives_country['value'],'-*')
    plt.title('Country: '+country)
    plt.xlabel('Years')
    plt.ylabel('Number')

interactive(children=(Dropdown(description='country', options=('Albania', 'Algeria', 'Angola', 'Argentina', 'A…

In [41]:
@interact
def show_beehives_for_geo_area(area=geo_areas):
    beehives_country=beehives.loc[beehives['country_or_area']==area]
    plt.plot(beehives_country['year'],beehives_country['value'],'-*')
    plt.title('Geographical area: '+area)
    plt.xlabel('Years')
    plt.ylabel('Number')

interactive(children=(Dropdown(description='area', options=('Africa +', 'Americas +', 'Asia +', 'Australia and…

In [42]:
@interact
def show_beehives_for_status_area(area=status_areas):
    beehives_country=beehives.loc[beehives['country_or_area']==area]
    plt.plot(beehives_country['year'],beehives_country['value'],'-*')
    plt.title('Status area: '+area)
    plt.xlabel('Years')
    plt.ylabel('Number')

interactive(children=(Dropdown(description='area', options=('LandLocked developing countries +', 'Least Develo…